# Time Series

Name: Koh June Wen

Admin Number: 2112956

Class: DAAAFT2A04

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab
import seaborn as sns
from datetime import timedelta

df = pd.read_csv('train.csv')
df.drop(columns=df.columns[-2:], inplace=True)
df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")
df = df.pivot(index=['Date', 'T', 'RH'], columns='Gas', values='Value')
df.reset_index(inplace=True)
df.set_index('Date', inplace=True)
df.info()

In [ ]:
test = pd.read_csv('test.csv')
test.drop(columns=test.columns[-2:], inplace=True)
test['Date'] = pd.to_datetime(test['Date'], format="%d/%m/%Y")
test['Value'] = np.nan
test = test.pivot(index=['Date', 'T', 'RH'], columns='Gas', values='Value')
test.reset_index(inplace=True)
test.set_index('Date', inplace=True)
test.info()

## Detecting Anomalies

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(data=df, x='Date', y='CO', label='CO')
sns.lineplot(data=df, x='Date', y='HC', label='HC')
sns.lineplot(data=df, x='Date', y='NO2', label='NO2')
sns.lineplot(data=df, x='Date', y='O3', label='O3')

In [ ]:
def percentage_change(values):
    previous_values = values.iloc[:-1]
    last_value = values.iloc[-1]

    percent_change = (last_value - np.mean(previous_values)) / np.mean(previous_values)

    return percent_change

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(20, 10))

df_pc_plot = df[['Date']]
df_pc_plot[['CO', 'HC', 'NO2', 'O3']] = df[['CO', 'HC', 'NO2', 'O3']].rolling(window=30).aggregate(percentage_change)

# CO
ax_CO = sns.lineplot(data=df, x='Date', y='CO', ax=ax[0, 0]).set(title='Original')
ax_CO_pc = sns.lineplot(data=df_pc_plot, x='Date', y='CO', ax=ax[0, 1]).set(title='Percentage Change')

# HC
ax_HC = sns.lineplot(data=df, x='Date', y='HC', ax=ax[1, 0])
ax_HC_pc = sns.lineplot(data=df_pc_plot, x='Date', y='HC', ax=ax[1, 1])

# NO2
ax_NO2 = sns.lineplot(data=df, x='Date', y='NO2', ax=ax[2, 0])
ax_NO2_pc = sns.lineplot(data=df_pc_plot, x='Date', y='NO2', ax=ax[2, 1])

# O3
ax_O3 = sns.lineplot(data=df, x='Date', y='O3', ax=ax[3, 0])
ax_O3_pc = sns.lineplot(data=df_pc_plot, x='Date', y='O3', ax=ax[3, 1])

## Cleaning of data
Cleaning the data makes the RMSE in the competition worse than not cleaning it.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
df_temp = df[df['T'] > 0]
df_temp

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(20, 10))
# df_temp.reset_index()
sns.lineplot(data=df_temp, x='Date', y='T', ax=ax[0])
sns.lineplot(data=df_temp, x='Date', y='RH', ax=ax[1])
sns.lineplot(data=df_temp, x='Date', y='CO', ax=ax[2])
sns.lineplot(data=df_temp, x='Date', y='HC', ax=ax[2])
sns.lineplot(data=df_temp, x='Date', y='NO2', ax=ax[2])
sns.lineplot(data=df_temp, x='Date', y='O3', ax=ax[2])

## Stationarity of Time Series Data
- Moving Average (Visual) - pd.rolling()
- Augmented Dickey-Fuller Test (Statistical) - adfuller()

In [ ]:
from statsmodels.tsa.stattools import acf, pacf, adfuller

def timeSeriesStationaryInfo(series, window):

    # Plot Rolling Statistics
    movingAverage = series.rolling(window).mean()
    movingStd = series.rolling(window).std()

    fig = plt.figure(figsize=(20, 10))

    orig = plt.plot(series, label='Original')
    ma = plt.plot(movingAverage, label='Moving Average')
    mstd = plt.plot(movingStd, label='Moving Standard Deviation')
    plt.title('Checking stationary of time series data by comparing original data with moving average and standard deviation')
    plt.legend(loc='best')
    plt.show()

    # Perform Dickey-Fuller Test
    print('Results from Dickey-Fuller Test')
    dftest = adfuller(series, autolag='AIC') # Find out abt AIC
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in dftest[4].items():
        dfoutput[f'Critical Value ({key})'] = value
    return dfoutput

### Cleaned

In [ ]:
T_temp_series = df_temp['T']
stationarity_T_temp = timeSeriesStationaryInfo(T_temp_series, 30)
stationarity_T_temp

#### Relative Humidity (RH)

In [ ]:
RH_temp_series = df_temp['RH']
stationarity_RH_temp = timeSeriesStationaryInfo(RH_temp_series, 30)
stationarity_RH_temp

#### CO

In [ ]:
CO_temp_series = df_temp['CO']
stationarity_CO_temp = timeSeriesStationaryInfo(CO_temp_series, 30)
stationarity_CO_temp

#### HC

In [ ]:
HC_temp_series = df_temp['HC']
stationarity_HC_temp = timeSeriesStationaryInfo(HC_temp_series, 30)
stationarity_HC_temp

#### NO2

In [ ]:
NO2_temp_series = df_temp['NO2']
stationarity_NO2_temp = timeSeriesStationaryInfo(NO2_temp_series, 30)
stationarity_NO2_temp

#### O3

In [ ]:
O3_temp_series = df_temp['O3']
stationarity_O3_temp = timeSeriesStationaryInfo(O3_temp_series, 30)
stationarity_O3_temp

### Original

#### Temperature (T)

In [ ]:
T_series = df[['T']]
stationarity_T = timeSeriesStationaryInfo(T_series, 30)
stationarity_T

#### Relative Humidity (RH)

In [ ]:
RH_series = df['RH']
stationarity_RH = timeSeriesStationaryInfo(RH_series, 30)
stationarity_RH

#### CO

In [ ]:
V_series = df['CO']
stationarity_CO = timeSeriesStationaryInfo(V_series, 30)
stationarity_CO

#### HC

In [ ]:
V_series = df['HC']
stationarity_HC = timeSeriesStationaryInfo(V_series, 30)
stationarity_HC

#### NO2

In [ ]:
V_series = df['NO2']
stationarity_NO2 = timeSeriesStationaryInfo(V_series, 30)
stationarity_NO2

#### O3

In [ ]:
V_series = df['O3']
stationarity_O3 = timeSeriesStationaryInfo(V_series, 30)
stationarity_O3

## Seasonal Decomposition
Purposes:
1. Visualising the components of the time series
2. Seasonal adjustment

`Multiplicative approach` - when the standard deviation of the time series data changes with time\
`Additive approach` - when the standard deviation remains constant\
If LOG(<em>value</em> / <em>value-1</em>) of all values in the dataframe has a normal distribution, the time series model is multiplicatiive

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# decompose_T_mul = seasonal_decompose(df['T'], model='multiplicative')
decompose_T = seasonal_decompose(df['T'], model='additive')
decompose_RH = seasonal_decompose(df['RH'], model='additive')
decompose_CO = seasonal_decompose(df['CO'], model='additive')
decompose_HC = seasonal_decompose(df['HC'], model='additive')
decompose_NO2 = seasonal_decompose(df['NO2'], model='additive')
decompose_O3 = seasonal_decompose(df['O3'], model='additive')

In [ ]:
seasonal_NO2 = decompose_T.seasonal
seasonal_NO2[seasonal_NO2 == seasonal_NO2[1]]

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 9)
decompose_T.plot()
plt.show()

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 9)
decompose_RH.plot()
plt.show()

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 9)
decompose_CO.plot()
plt.show()

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 9)
decompose_HC.plot()
plt.show()

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 9)
decompose_NO2.plot()
plt.show()

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 9)
decompose_O3.plot()
plt.show()

## Checking for seasonality
Presence of seasonality in a time series model will be proven if a periodogram graph has an obvious spike in the y value while the surround values are relatively flat

In [ ]:
from scipy.signal import periodogram

In [ ]:
T_f, T_den = periodogram(df['T'], 328)
plt.semilogy(T_f, T_den)
plt.ylim([1e-1, 1e2])
plt.xlabel('Frequency (Hz)')
plt.show()

In [ ]:
RH_f, RH_den = periodogram(df['RH'], 328)
plt.semilogy(RH_f, RH_den)
plt.ylim([1e-1, 1e2])
plt.xlabel('Frequency (Hz)')
plt.show()

In [ ]:
CO_f, CO_den = periodogram(df['CO'], 328)
plt.semilogy(CO_f, CO_den)
plt.ylim([1e1, 1e4])
plt.xlabel('Frequency (Hz)')
plt.show()

In [ ]:
HC_f, HC_den = periodogram(df['HC'], 328)
plt.semilogy(HC_f, HC_den)
plt.ylim([1e1, 1e4])
plt.xlabel('Frequency (Hz)')
plt.show()

In [ ]:
NO2_f, NO2_den = periodogram(df['NO2'], 328)
plt.semilogy(NO2_f, NO2_den)
plt.ylim([1e1, 1e4])
plt.xlabel('Frequency (Hz)')
plt.show()

In [ ]:
O3_f, O3_den = periodogram(df['O3'], 328)
plt.semilogy(O3_f, O3_den)
plt.ylim([1e1, 1e4])
plt.xlabel('Frequency (Hz)')
plt.show()

## Causality Test

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

maxlag = 30
variables=df.columns  
matrix = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
for col in matrix.columns:
    for row in matrix.index:
        test_result = grangercausalitytests(df[[row, col]], maxlag=maxlag, verbose=False)            
        p_values = [round(test_result[i+1][0]['ssr_chi2test'][1],4) for i in range(maxlag)] # Chi-square test - test for association between 2 variables       
        min_p_value = np.min(p_values)
        matrix.loc[row, col] = min_p_value
matrix.columns = [var + '_x' for var in variables]
matrix.index = [var + '_y' for var in variables]
print(matrix)

# From the result, each column represents a predictor x of each variable and each row
# represents the response y and the p-value of each pair of variabble are shown in the
# matrix

# E.g. Take the value 0.0012 (in row 3, column 4), it refers that HC_x is causal to CO_y
# and hence, we can reject the null hypothesis

rej_null_count = matrix[matrix < 0.05].count().sum()
print(f"\n{rej_null_count / (len(df.columns) ** 2 - len(df.columns)) * 100}% of the comparisons are less than 0.05. This suggests that most of the variables are interchangably causing each other.")
print(f"This makes this system of multi time series a good candidate for using VAR models to forecast.")

## Cointegration Test
This helps to establish the presence of a statistically significant connection between two or more series.\
When two or more time series are cointegrated, it means that they have a long run, statistically significant relationship.\
`How it works`: When you have two or more time series, and there exists a linear combination of them that has an `order of integration` (the number of differencing required to make a non-stationary time series stationary) less than that of the individual series, then the collection of series is said to be cointegrated.

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(df)

## Making time series data stationary
Purpose: It can be easier to model. Statistical modeling methods assume or require the time series to be stationary\
\
Ways to convert non-stationary to stationary:
- Log transforming of the data (If the time series is a multiplicative model)
- Taking the square root of the data
- Taking the cube root
- Proportional change

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adfuller_test(series, signif=0.05, name='', verbose=False):
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

In [ ]:
df_temp_diff = df_temp.diff().dropna()

for name, column in df_temp_diff.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

# all the series are already stationary so there is no need to differentiate them

In [ ]:
df_diff = df.diff().dropna()
for name, column in df_diff.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')


## Model Building
`AIC`: penalizes complex models less, meaning that it may put more emphasis on model performance on the training dataset, and, in turn, select more complex models.\
`BIC`: penalizes the model more for its complexity, meaning that more complex models will have a worse (larger) score and will, in turn, be less likely to be selected.\
`Endogenous variables`: values that are determined by other variables in the system (Dependent variable)\
`Exogenous variables`: a variable that is not affected by other variables in the system (Independent variable)

In [ ]:
from statsmodels.tsa.statespace.sarimax import  SARIMAX
from statsmodels.tsa.arima.model import ARIMA, ARIMAResults
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from statsmodels.tsa.api import VAR, VARMAX

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook

tss = TimeSeriesSplit(n_splits=3, test_size=30)

train_df, test_df = [], []

for train_i, test_i in tss.split(df):
    train_df.append(df.iloc[train_i])
    test_df.append(df.iloc[test_i])

train_df.append(df)
test_df.append(test)

train_df_diff, test_df_diff = [], []

for train_i, test_i in tss.split(df_diff):
    train_df_diff.append(df_diff.iloc[train_i])
    test_df_diff.append(df_diff.iloc[test_i])

train_df_diff.append(df_diff)
test_df_diff.append(test[['T', 'RH']].diff().dropna())

In [ ]:
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning
# warnings.simplefilter('ignore', ValueWarning)

# with warnings.catch_warnings():
warnings.filterwarnings('ignore')

### VAR
Can be used when 2 or more time series influence each other

#### Optimising the number of lags

In [ ]:
results_dict = {"AIC": [], "BIC": [], "FPE": [], "HQIC": []}

v = 3 # 0 - 1st validation, 1 - 2nd validation, 2 - 3rd validation, 3 - final test

exog_df = train_df[v].iloc[:, :2]
endog_df = train_df[v].iloc[:, 2:]

# exog_df = df.iloc[:, :2]
# endog_df = df.iloc[:, 2:]

var_model = VAR(exog=exog_df, endog=endog_df)
for i in range(1, 20):
    result = var_model.fit(i)
    results_dict["AIC"].append(result.aic)
    results_dict["BIC"].append(result.bic)
    results_dict["FPE"].append(result.fpe)
    results_dict["HQIC"].append(result.hqic)
    print(f"Lag Order = {i}")
    print(f"AIC : {result.aic}")
    print(f"BIC : {result.bic}")
    print(f"FPE : {result.fpe}")
    print(f"HQIC : {result.hqic} \n")

In [ ]:
scores = []

for k in tqdm(range(1, 20)):
    rmse_arr = []
    aic_arr = []
    for vt in range(3):
        exog_df = train_df[vt].iloc[:, :2]
        endog_df = train_df[vt].iloc[:, 2:]
        n = len(test_df[vt].index)
        exog_future = test_df[vt][['T', 'RH']]

        var_model = VAR(exog=exog_df, endog=endog_df)
        result = var_model.fit(k)
        var_lag_order = result.k_ar
        forecast_input = endog_df.values[-var_lag_order:]
        fc = result.forecast(y=forecast_input, steps=n, exog_future=exog_future)
        fc_df = pd.DataFrame(fc, columns=endog_df.columns, index=test_df[vt].index)
        fc_df_melt = pd.melt(fc_df, ignore_index=False)
        test_df_melt = pd.melt(test_df[vt].drop(columns=['T', 'RH']), ignore_index=False)
        rmse_arr.append(mean_squared_error(fc_df_melt['value'], test_df_melt['value']) ** 0.5)
        aic_arr.append(result.aic)
    scores.append([k, np.mean(aic_arr), np.mean(rmse_arr)])

scores_df = pd.DataFrame(scores, columns=['k', 'aic', 'rmse'])
print("rmse : \n", scores_df.sort_values(by='rmse', ascending=True).head(1))
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))

In [ ]:
x = np.arange(1, 20)

fig, ax = plt.subplots(2, 2, figsize=(20, 10))

ax[0,0].plot(x, results_dict['AIC'])
ax[0,0].set_title('AIC')
ax[0,1].plot(x, results_dict['BIC'])
ax[0,1].set_title('BIC')
ax[1,0].plot(x, results_dict['FPE'])
ax[1,0].set_title('FPE')
ax[1,1].plot(x, results_dict['HQIC'])
ax[1,1].set_title('HQIC')
plt.show()

#### Model Fitting

In [ ]:
var_model_fitted = var_model.fit(1)
var_model_fitted.summary()

#### Checking for Serial Correlation of Residuals (Errors)
Used to check if there is any leftover pattern in the residuals (errors).\
If there is any correlation left in the residuals, then, there is some pattern in the time series that is still left to be explained by the model.\
Checking for Serial Correlation is to ensure that the model is sufficiently able to explain the variances and patterns in the time series.\
\
The value can of this statistic can vary between 0 and 4.\
The closer it is to the value 2, then there is no significant serial correlation.\
The closer it is to the value 0, then there is a positive serial correlation.\
The closer it is to the value 4, then there is a negative serial correlation.

In [ ]:
from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(var_model_fitted.resid)

def adjust(val, length= 6): return str(val).ljust(length)

for col, val in zip(df.columns, out):
    print(adjust(col), ':', round(val, 2))

# As the values are extremely close to the value 2, there is no significant serial correlation in the residuals.

#### Forecasting

In [ ]:
var_lag_order = var_model_fitted.k_ar

forecast_input = endog_df.values[-var_lag_order:]
forecast_input

endog_df.iloc[-var_lag_order:]

In [ ]:
n = len(test_df[v].index)
exog_future = test_df[v][['T', 'RH']]

fc = var_model_fitted.forecast(y=forecast_input, steps=n, exog_future=exog_future)
fc_df = pd.DataFrame(fc, columns=endog_df.columns, index=test_df[v].index)
fc_df

In [ ]:
sns.lineplot(data=fc_df, x='Date', y='CO', label='CO')
sns.lineplot(data=fc_df, x='Date', y='HC', label='HC')
sns.lineplot(data=fc_df, x='Date', y='NO2', label='NO2')
sns.lineplot(data=fc_df, x='Date', y='O3', label='O3')

In [ ]:
## RMSE check validation
pred_fc = pd.melt(fc_df, ignore_index=False)
true_fc = pd.melt(test_df[v].drop(columns=['T', 'RH']), ignore_index=False)
print(mean_squared_error(pred_fc['value'], true_fc['value']) ** 0.5)

`First` validation check RMSE (No. Lags - `8`): 187.26582\
`Second` validation check RMSE (No. Lags - `8`): 188.50737\
`Third` validation check RMSE (No. Lags - `18`): 201.34537\
`Third` validation check RMSE (No. Lags - `8`): 177.59665\
Avg RMSE from validation checks: 184.45661

In [ ]:
fc_df_submit = pd.melt(fc_df, ignore_index=False)
fc_df_submit.reset_index(inplace=True)
fc_df_submit.insert(0, 'id', fc_df_submit.index)
fc_df_submit.drop(columns=['Date', 'Gas'], inplace=True)
fc_df_submit.to_csv('VAR_predictions.csv', index=False)

###  (No. Lags - 17) RMSE 195.91798 ###
### (No. Lags - 8) RMSE 174.00740 ###

#### Diffed

In [ ]:
results_diff_dict = {"AIC": [], "BIC": [], "FPE": [], "HQIC": []}

v = 3

exog_df_diff = train_df_diff[v].iloc[:, :2]
endog_df_diff = train_df_diff[v].iloc[:, 2:]

var_model_diff = VAR(exog=exog_df_diff, endog=endog_df_diff)
for i in range(1, 20):
    result = var_model_diff.fit(i)
    results_diff_dict["AIC"].append(result.aic)
    results_diff_dict["BIC"].append(result.bic)
    results_diff_dict["FPE"].append(result.fpe)
    results_diff_dict["HQIC"].append(result.hqic)
    print(f"Lag Order = {i}")
    print(f"AIC : {result.aic}")
    print(f"BIC : {result.bic}")
    print(f"FPE : {result.fpe}")
    print(f"HQIC : {result.hqic} \n")

In [ ]:
x = np.arange(1, 20)

fig, ax = plt.subplots(2, 2, figsize=(20, 10))

ax[0,0].plot(x, results_diff_dict['AIC'])
ax[0,0].set_title('AIC')
ax[0,1].plot(x, results_diff_dict['BIC'])
ax[0,1].set_title('BIC')
ax[1,0].plot(x, results_diff_dict['FPE'])
ax[1,0].set_title('FPE')
ax[1,1].plot(x, results_diff_dict['HQIC'])
ax[1,1].set_title('HQIC')
plt.show()

In [ ]:
var_lags_orders_diff = var_model_diff.select_order(maxlags=19)
var_lags_orders_diff.summary()

In [ ]:
var_model_diff_fitted = var_model_diff.fit(7, ic='aic')
var_model_diff_fitted.summary()

In [ ]:
from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(var_model_diff_fitted.resid)

def adjust(val, length= 6): return str(val).ljust(length)

for col, val in zip(df.columns, out):
    print(adjust(col), ':', round(val, 2))

In [ ]:
var_lag_order_diff = var_model_diff_fitted.k_ar

forecast_input_diff = endog_df_diff.values[-var_lag_order_diff:]
forecast_input_diff

endog_df_diff.iloc[-var_lag_order_diff:]

In [ ]:
exog_future_diff = test_df_diff[v][['T', 'RH']]
n = len(exog_future_diff.index)

fc_diff = var_model_diff_fitted.forecast(y=forecast_input_diff, steps=n, exog_future=exog_future_diff)
fc_diff_df = pd.DataFrame(fc_diff, columns=endog_df_diff.columns, index=exog_future_diff.index)
fc_diff_df

In [ ]:
fc_undiff_df = fc_diff_df.cumsum() + df.iloc[-1]
first_record = (fc_diff_df.iloc[0] + df.iloc[-1]).to_frame().reset_index()
first_record.rename(columns={first_record.columns[1]: "value"}, inplace=True)
first_record['Date'] = (pd.to_datetime(fc_undiff_df.index[0]) - timedelta(days=1)).strftime("%Y-%m-%d")
first_record = pd.pivot(first_record, index='Date', columns='Gas', values='value')
fc_undiff_df = pd.concat([first_record, fc_undiff_df], axis=0)
fc_undiff_df.drop(columns=['T', 'RH'], inplace=True)

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 9)
sns.lineplot(data=fc_undiff_df, x='Date', y='CO', label='CO')
sns.lineplot(data=fc_undiff_df, x='Date', y='HC', label='HC')
sns.lineplot(data=fc_undiff_df, x='Date', y='NO2', label='NO2')
sns.lineplot(data=fc_undiff_df, x='Date', y='O3', label='O3')

In [ ]:
## RMSE check validation
pred_fc = pd.melt(fc_diff_df, ignore_index=False)
true_fc = pd.melt(test_df_diff[v].drop(columns=['T', 'RH']), ignore_index=False)
print(mean_squared_error(pred_fc['value'], true_fc['value']) ** 0.5)

`First` validation check RMSE (No. Lags - `7`): 179.54453\
`Second` validation check RMSE (No. Lags - `7`): 168.06703\
`Third` validation check RMSE (No. Lags - `7`): 173.83961\
Avg RMSE from validation checks: 173.81706

In [ ]:
fc_df_submit = pd.melt(fc_undiff_df, ignore_index=False)
fc_df_submit.reset_index(inplace=True)
fc_df_submit.insert(0, 'id', fc_df_submit.index)
fc_df_submit.drop(columns=['Date', 'Gas'], inplace=True)
fc_df_submit.to_csv('VAR_predictions.csv', index=False)

### RMSE 212.50707 ###

### ARIMA

#### ACF & PACF

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 7), sharey=True)
acf_ = plot_acf(df['T'], ax=ax[0])
pacf_ = plot_pacf(df['T'], ax=ax[1])
plt.ylim([-1.4, 1.4])
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 7), sharey=True)
acf_ = plot_acf(df['RH'], ax=ax[0])
pacf_ = plot_pacf(df['RH'], ax=ax[1])
plt.ylim([-1.4, 1.4])
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 7), sharey=True)
acf_ = plot_acf(df['CO'], ax=ax[0])
pacf_ = plot_pacf(df['CO'], ax=ax[1])
plt.ylim([-1.4, 1.4])
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 7), sharey=True)
acf_ = plot_acf(df['HC'], ax=ax[0])
pacf_ = plot_pacf(df['HC'], ax=ax[1])
plt.ylim([-1.4, 1.4])
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 7), sharey=True)
df_NO2 = (df['NO2'] - df['NO2'].rolling(30).mean()).dropna()
acf_ = plot_acf(df['NO2'], ax=ax[0])
pacf_ = plot_pacf(df['NO2'], ax=ax[1])

# acf_ = plot_acf(df_NO2, ax=ax[0])
# pacf_ = plot_pacf(df_NO2, ax=ax[1])

plt.ylim([-1.4, 1.4])
plt.show()

## Shows a seasonal order of 8 for NO2

In [ ]:
df_NO2_diff = df['NO2'].diff(8).dropna()

fig, ax = plt.subplots(1, 2, figsize=(20, 7), sharey=True)
acf_ = plot_acf(df_NO2_diff, lags=[8, 16,24,32,40,48,56,64], ax=ax[0])
pacf_ = plot_pacf(df_NO2_diff, lags=[8, 16,24,32,40,48,56,64], ax=ax[1])
plt.ylim([-1.4, 1.4])
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 7), sharey=True)
acf_ = plot_acf(df['O3'], ax=ax[0])
pacf_ = plot_pacf(df['O3'], ax=ax[1])
plt.ylim([-1.4, 1.4])
plt.show()

#### Modeling Tuning
Using AIC for comparison instead of BIC because AIC is better for modeling a predictive model while BIC is better for modeling an explanatory model

In [ ]:
v = 3
exog_df = train_df[v].iloc[:, :2]
endog_df = train_df[v].iloc[:, 2:]

n = len(test_df[v].index)

##### CO

In [ ]:
# cross validation
scores = []

order_arr = []

for q in range(1,3):
    for d in range(3):
        for p in range(1,3):
            order_arr.append([q,d,p])

for order in tqdm(order_arr):
    rmse_arr = []
    aic_arr = []
    bic_arr = []
    for v in range(3):
        exog_df = train_df[v].iloc[:, :2]
        endog_df = train_df[v].iloc[:, 2:]
        n = len(test_df[v].index)

        arima_model = ARIMA(endog=endog_df['CO'], exog=exog_df, order=order)
        arima_result = arima_model.fit()
        fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
        rmse_arr.append(mean_squared_error(fc, test_df[v]['CO']) ** 0.5)
        aic_arr.append(arima_result.aic)
        bic_arr.append(arima_result.bic)
    
    scores.append([order[0], order[1], order[2], np.mean(aic_arr), np.mean(bic_arr), np.mean(rmse_arr)])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'aic', 'bic', 'rmse'])
print("rmse : \n", scores_df.sort_values(by='rmse', ascending=True).head(1))
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))

In [ ]:
# no cross validation
scores = []

order_arr = []

for q in range(1,3):
    for d in range(3):
        for p in range(1,3):
            order_arr.append([q,d,p])

for order in tqdm(order_arr):
    exog_df = train_df[v].iloc[:, :2]
    endog_df = train_df[v].iloc[:, 2:]
    n = len(test_df[v].index)

    arima_model = ARIMA(endog=endog_df['CO'], exog=exog_df, order=order)
    arima_result = arima_model.fit()
    fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
    scores.append([order[0], order[1], order[2], arima_result.aic, arima_result.bic])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'aic', 'bic'])
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))

In [ ]:
scores = []

order_arr = []

for q in range(1,3):
    for d in range(3):
        for p in range(1,3):
            for Q in range(1,3):
                for D in range(3):
                    for P in range(1,3):
                        order_arr.append([[q,d,p], [Q,D,P,8]])

for order in tqdm(order_arr):
    exog_df = train_df[v].iloc[:, :2]
    endog_df = train_df[v].iloc[:, 2:]
    n = len(test_df[v].index)

    arima_model = ARIMA(endog=endog_df['CO'], exog=exog_df, order=order[0], seasonal_order=order[1])
    arima_result = arima_model.fit()
    fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
    scores.append([order[0][0], order[0][1], order[0][2], order[1][0], order[1][1], order[1][2], arima_result.aic, arima_result.bic])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'P', 'D', 'Q', 'aic', 'bic'])
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))

##### HC

In [ ]:
# cross validation
scores = []

order_arr = []

for q in range(1,3):
    for d in range(3):
        for p in range(1,3):
            order_arr.append([q,d,p])

for order in tqdm(order_arr):
    rmse_arr = []
    aic_arr = []
    bic_arr = []
    for v in range(3):
        exog_df = train_df[v].iloc[:, :2]
        endog_df = train_df[v].iloc[:, 2:]
        n = len(test_df[v].index)

        arima_model = ARIMA(endog=endog_df['HC'], exog=exog_df, order=order)
        arima_result = arima_model.fit()
        fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
        rmse_arr.append(mean_squared_error(fc, test_df[v]['HC']) ** 0.5)
        aic_arr.append(arima_result.aic)
        bic_arr.append(arima_result.bic)
    
    scores.append([order[0], order[1], order[2], np.mean(aic_arr), np.mean(bic_arr), np.mean(rmse_arr)])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'aic', 'bic', 'rmse'])
print("rmse : \n", scores_df.sort_values(by='rmse', ascending=True).head(1))
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))

# no cross validation
for order in tqdm(order_arr):
    exog_df = train_df[v].iloc[:, :2]
    endog_df = train_df[v].iloc[:, 2:]
    n = len(test_df[v].index)

    arima_model = ARIMA(endog=endog_df['HC'], exog=exog_df, order=order)
    arima_result = arima_model.fit()
    fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
    scores.append([order[0], order[1], order[2], arima_result.aic, arima_result.bic])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'aic', 'bic'])
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))

In [ ]:
# no cross validation
scores = []

order_arr = []

for q in range(1,3):
    for d in range(3):
        for p in range(1,3):
            order_arr.append([q,d,p])

for order in tqdm(order_arr):
    exog_df = train_df[v].iloc[:, :2]
    endog_df = train_df[v].iloc[:, 2:]
    n = len(test_df[v].index)

    arima_model = ARIMA(endog=endog_df['HC'], exog=exog_df, order=order)
    arima_result = arima_model.fit()
    fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
    scores.append([order[0], order[1], order[2], arima_result.aic, arima_result.bic])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'aic', 'bic'])
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))

In [ ]:
scores = []

order_arr = []

for q in range(1,3):
    for d in range(3):
        for p in range(1,3):
            for Q in range(1,3):
                for D in range(3):
                    for P in range(1,3):
                        order_arr.append([[q,d,p], [Q,D,P,8]])

for order in tqdm(order_arr):
    exog_df = train_df[v].iloc[:, :2]
    endog_df = train_df[v].iloc[:, 2:]
    n = len(test_df[v].index)

    arima_model = ARIMA(endog=endog_df['HC'], exog=exog_df, order=order[0], seasonal_order=order[1])
    try:
        arima_result = arima_model.fit()
    except: print(order)
    fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
    scores.append([order[0][0], order[0][1], order[0][2], order[1][0], order[1][1], order[1][2], arima_result.aic, arima_result.bic])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'P', 'D', 'Q', 'aic', 'bic'])
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))

##### NO2

In [ ]:
# cross validation
scores = []

order_arr = []

for q in range(1,3):
    for d in range(3):
        for p in range(1,3):
            order_arr.append([q,d,p])

for order in tqdm(order_arr):
    rmse_arr = []
    aic_arr = []
    bic_arr = []
    for v in range(3):
        exog_df = train_df[v].iloc[:, :2]
        endog_df = train_df[v].iloc[:, 2:]
        n = len(test_df[v].index)

        arima_model = ARIMA(endog=endog_df['NO2'], exog=exog_df, order=order)
        arima_result = arima_model.fit()
        fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
        rmse_arr.append(mean_squared_error(fc, test_df[v]['NO2']) ** 0.5)
        aic_arr.append(arima_result.aic)
        bic_arr.append(arima_result.bic)
    
    scores.append([order[0], order[1], order[2], np.mean(aic_arr), np.mean(bic_arr), np.mean(rmse_arr)])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'aic', 'bic', 'rmse'])
print("rmse : \n", scores_df.sort_values(by='rmse', ascending=True).head(1))
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))

In [ ]:
# no cross validation
scores = []

order_arr = []

for q in range(1,3):
    for d in range(3):
        for p in range(1,3):
            order_arr.append([q,d,p])

for order in tqdm(order_arr):
    exog_df = train_df[v].iloc[:, :2]
    endog_df = train_df[v].iloc[:, 2:]
    n = len(test_df[v].index)

    arima_model = ARIMA(endog=endog_df['NO2'], exog=exog_df, order=order)
    arima_result = arima_model.fit()
    fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
    scores.append([order[0], order[1], order[2], arima_result.aic, arima_result.bic])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'aic', 'bic'])
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))

In [ ]:
# cross validation
scores = []

order_arr = []

for q in range(1,3):
    for d in range(3):
        for p in range(1,3):
            for Q in range(1,3):
                for D in range(3):
                    for P in range(1,3):
                        order_arr.append([[q,d,p], [Q,D,P,7]])

for order in tqdm(order_arr):
    rmse_arr = []
    aic_arr = []
    bic_arr = []
    for v in range(3):
        exog_df = train_df[v].iloc[:, :2]
        endog_df = train_df[v].iloc[:, 2:]
        n = len(test_df[v].index)

        try:
            arima_model = ARIMA(endog=endog_df['NO2'], exog=exog_df, order=order[0], seasonal_order=order[1])
            arima_result = arima_model.fit()
            fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
            rmse_arr.append(mean_squared_error(fc, test_df[v]['NO2']) ** 0.5)
            aic_arr.append(arima_result.aic)
            bic_arr.append(arima_result.bic)
        except: continue
    
    scores.append([order[0][0], order[0][1], order[0][2], order[1][0], order[1][1], order[1][2], np.mean(aic_arr), np.mean(bic_arr), np.mean(rmse_arr)])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'P', 'D', 'Q', 'aic', 'bic', 'rmse'])
print("rmse : \n", scores_df.sort_values(by='rmse', ascending=True).head(1))
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))

In [ ]:
# no cross validation
scores = []

order_arr = []

for q in range(1,3):
    for d in range(3):
        for p in range(1,3):
            for Q in range(1,3):
                for D in range(3):
                    for P in range(1,3):
                        order_arr.append([[q,d,p], [Q,D,P,8]])

for order in tqdm(order_arr):
    exog_df = train_df[v].iloc[:, :2]
    endog_df = train_df[v].iloc[:, 2:]
    n = len(test_df[v].index)

    arima_model = ARIMA(endog=endog_df['NO2'], exog=exog_df, order=order[0], seasonal_order=order[1])
    arima_result = arima_model.fit()
    fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
    scores.append([order[0][0], order[0][1], order[0][2], order[1][0], order[1][1], order[1][2], arima_result.aic, arima_result.bic])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'P', 'D', 'Q', 'aic', 'bic'])
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))

##### O3

In [ ]:
# cross validation
scores = []

order_arr = []

for q in range(1,3):
    for d in range(3):
        for p in range(1,3):
            order_arr.append([q,d,p])

for order in tqdm(order_arr):
    rmse_arr = []
    aic_arr = []
    bic_arr = []
    for v in range(3):
        exog_df = train_df[v].iloc[:, :2]
        endog_df = train_df[v].iloc[:, 2:]
        n = len(test_df[v].index)

        arima_model = ARIMA(endog=endog_df['O3'], exog=exog_df, order=order)
        arima_result = arima_model.fit()
        fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
        rmse_arr.append(mean_squared_error(fc, test_df[v]['O3']) ** 0.5)
        aic_arr.append(arima_result.aic)
        bic_arr.append(arima_result.bic)
    
    scores.append([order[0], order[1], order[2], np.mean(aic_arr), np.mean(bic_arr), np.mean(rmse_arr)])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'aic', 'bic', 'rmse'])
print("rmse : \n", scores_df.sort_values(by='rmse', ascending=True).head(1))
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))



In [ ]:
# no cross validation
scores = []

order_arr = []

for q in range(1,3):
    for d in range(3):
        for p in range(1,3):
            order_arr.append([q,d,p])

for order in tqdm(order_arr):
    exog_df = train_df[v].iloc[:, :2]
    endog_df = train_df[v].iloc[:, 2:]
    n = len(test_df[v].index)

    arima_model = ARIMA(endog=endog_df['NO2'], exog=exog_df, order=order)
    arima_result = arima_model.fit()
    fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
    scores.append([order[0], order[1], order[2], arima_result.aic, arima_result.bic])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'aic', 'bic'])
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))

In [ ]:
scores = []

order_arr = []

for q in range(1,3):
    for d in range(3):
        for p in range(1,3):
            for Q in range(1,3):
                for D in range(3):
                    for P in range(1,3):
                        order_arr.append([[q,d,p], [Q,D,P,8]])

for order in tqdm(order_arr):
    exog_df = train_df[v].iloc[:, :2]
    endog_df = train_df[v].iloc[:, 2:]
    n = len(test_df[v].index)

    arima_model = ARIMA(endog=endog_df['O3'], exog=exog_df, order=order[0], seasonal_order=order[1])
    try:
        arima_result = arima_model.fit()
    except:print(order)
    fc = arima_result.forecast(steps=n, exog=test_df[v][['T', 'RH']])
    scores.append([order[0][0], order[0][1], order[0][2], order[1][0], order[1][1], order[1][2], arima_result.aic, arima_result.bic])

scores_df = pd.DataFrame(scores, columns=['p', 'd', 'q', 'P', 'D', 'Q', 'aic', 'bic'])
print("aic : \n", scores_df.sort_values(by='aic', ascending=True).head(1))
print("bic : \n", scores_df.sort_values(by='bic', ascending=True).head(1))

#### Model Building

In [ ]:
## validation (AIC) # with HC,NO2 seasonal order
CO_order = (1,1,2)
HC_order, HC_seasonal_order = (2,1,1), (2,2,2,8)
NO2_order, NO2_seasonal_order = (2,1,1), (1,2,2,8)
O3_order = (2,2,1)

## validation (AIC) # with cross validation
# CO_order = (2,1,1)
# HC_order = (2,1,1)
# NO2_order, NO2_seasonal_order = (1,0,2), (2,2,2,8)
# O3_order = (2,1,1)

## validation (RMSE) # with cross validation
# CO_order = (2,0,1)
# HC_order = (2,0,1)
# NO2_order, NO2_seasonal_order = (1,0,1), (1,0,2,8)
# O3_order = (2,0,2)

## validation (RMSE) # with NO2 seasonal order
# CO_order = (2,2,1)
# HC_order = (1,1,2)
# NO2_order, NO2_seasonal_order = (2,2,2), (2,1,2,8)
# O3_order = (1,0,1)

## validation (RMSE)
# CO_order = (2,2,1)
# HC_order = (1,1,2)
# NO2_order = (2,2,2)
# O3_order = (1,0,1)

## validation (AIC)
# CO_order = (2,1,1)
# HC_order = (2,1,1)
# NO2_order, NO2_seasonal_order = (2,1,1), (1,2,2,8)
# O3_order = (2,1,1)

## validation (AIC) # no cross validation | with NO2 seasonal order
# CO_order = (1,1,2)
# HC_order = (2,1,1)
# NO2_order, NO2_seasonal_order = (2,1,1), (1,2,2,8)
# O3_order = (1,1,2)

## validation (AIC) # no cross validation | no No2 seasonal order
# CO_order = (1,1,2)
# HC_order = (2,1,1)
# NO2_order = (2,1,1)
# O3_order = (2,1,1)

v = 3
exog_df = train_df[v].iloc[:, :2]
endog_df = train_df[v].iloc[:, 2:]

n = len(test_df[v].index)

arima_model_CO = ARIMA(endog=endog_df['CO'], exog=exog_df, order=CO_order)
# arima_model_CO = ARIMA(endog=endog_df['CO'], exog=exog_df, order=CO_order, seasonal_order=CO_seasonal_order)
arima_result_CO = arima_model_CO.fit()

# arima_model_HC = ARIMA(endog=endog_df['HC'], exog=exog_df, order=HC_order)
arima_model_HC = ARIMA(endog=endog_df['HC'], exog=exog_df, order=HC_order, seasonal_order=HC_seasonal_order)
arima_result_HC = arima_model_HC.fit()

# arima_model_NO2 = ARIMA(endog=endog_df['NO2'], exog=exog_df, order=NO2_order)
arima_model_NO2 = ARIMA(endog=endog_df['NO2'], exog=exog_df, order=NO2_order, seasonal_order=NO2_seasonal_order)
arima_result_NO2 = arima_model_NO2.fit()

arima_model_O3 = ARIMA(endog=endog_df['O3'], exog=exog_df, order=O3_order)
# arima_model_O3 = ARIMA(endog=endog_df['O3'], exog=exog_df, order=O3_order, seasonal_order=O3_seasonal_order)
arima_result_O3 = arima_model_O3.fit()

#### Model Diagonstics

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 9)

##### CO

In [ ]:
arima_result_CO.plot_diagnostics()
plt.show()

# From these plots we can observe that the residuals conform to normal distribution
# The correlogram suggests that there is no auto correlation in the residuals

In [ ]:
arima_result_CO.summary()

##### HC

In [ ]:
arima_result_HC.plot_diagnostics()
plt.show()

In [ ]:
arima_result_HC.summary()

##### NO2
Without adding a seasonal order for NO2, diagnostic plot and summary shows that the residuals are non-normal which means that the prediction intervals will not be accurate.\
Adding a season order decreases the Jarque-Bera (a metric to determine the normality) value significantly. However, the resdiuals are still non-normal

In [ ]:
arima_result_NO2.plot_diagnostics()
plt.show()

## the NO2 model is not optimised
## Histogram does not fit a normal distribution
## The residuals of the Q-Q graph is not following the straight line
## Correlogram there is a significant correlation at value 7

In [ ]:
arima_result_NO2.summary()

##### O3

In [ ]:
arima_result_O3.plot_diagnostics()
plt.show()

In [ ]:
arima_result_O3.summary()

#### Forecasting

In [ ]:
fc_HC = arima_result_HC.forecast(steps=n, exog=test_df[v][['T', 'RH']])
fc_CO = arima_result_CO.forecast(steps=n, exog=test_df[v][['T', 'RH']])
fc_NO2 = arima_result_NO2.forecast(steps=n, exog=test_df[v][['T', 'RH']])
fc_O3 = arima_result_O3.forecast(steps=n, exog=test_df[v][['T', 'RH']])

In [ ]:
final_fc = pd.DataFrame(pd.concat([fc_CO, fc_HC, fc_NO2, fc_O3], axis=0, ignore_index=False))
final_fc

In [ ]:
pd.melt(test_df[v].drop(columns=['T', 'RH']))['value']

In [ ]:
print(f"RMSE : {mean_squared_error(final_fc, pd.melt(test_df[v].drop(columns=['T', 'RH']))['value']) ** 0.5}")

### Using RMSE as determining metric ###
# 166.

### Using RMSE with seasonal order for NO2 ###
# 163.57124

### Using AIC as determining metric ###
# 169.84430

In [ ]:
fc_df = pd.concat([fc_CO, fc_HC, fc_NO2, fc_O3], axis=1)
fc_df.columns = ['CO', 'HC', 'NO2', 'O3']
fc_df.index.name = 'Date'
sns.lineplot(data=fc_df, x='Date', y='CO', label='CO')
sns.lineplot(data=fc_df, x='Date', y='HC', label='HC')
sns.lineplot(data=fc_df, x='Date', y='NO2', label='NO2')
sns.lineplot(data=fc_df, x='Date', y='O3', label='O3')

In [ ]:
fc_df_submit = pd.melt(fc_df, ignore_index=False)
fc_df_submit.reset_index(inplace=True)
fc_df_submit.insert(0, 'id', fc_df_submit.index)
fc_df_submit.drop(columns=['Date', 'variable'], inplace=True)
fc_df_submit.to_csv('ARIMA_indiv_predictions.csv', index=False)

### RMSE 154.24407 ### NO2 no seasonal order AIC metric
### RMSE 153.33767 ### NO2 with seasonal order AIC metric
### RMSE 183.41228 ### NO2 with seasonal order RMSE metric
### RMSE 153.53418 ### RMSE | cross validation | NO2 seasonal order
### RMSE 176.88901 ### AIC | cross validation | NO2 seasonal order
### RMSE 194.33847 ### AIC | HC,NO2 seasonal order

# Using RMSE as gridsearch metric may have overfitted the model